# Day 3: Feature Engineering for Fraud Detection

**Advanced Feature Engineering Techniques for Transaction Fraud Detection**

## Overview
- **Objective**: Learn advanced feature engineering for fraud detection
- **Techniques**: Velocity features, Deviation features, Merchant encoding
- **Goal**: Improve model performance with engineered features

## What You'll Learn
1. **Velocity Features**: Transaction frequency and amounts over time windows
2. **Deviation Features**: Compare current behavior to historical patterns
3. **Merchant Features**: Target encoding and risk profiling
4. **Feature Selection**: SHAP-based importance ranking

---

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported!")

## 2. Generate Sample Transaction Data

In [ ]:
# Generate synthetic transaction data
np.random.seed(42)

# Create 10000 transactions across 100 users
n_transactions = 10000
n_users = 100

# Generate timestamps over 30 days
base_date = pd.Timestamp('2024-01-01')
timestamps = [base_date + pd.Timedelta(days=np.random.randint(0, 30), 
                                       hours=np.random.randint(0, 24), 
                                       minutes=np.random.randint(0, 60)) 
              for _ in range(n_transactions)]

df = pd.DataFrame({
    'user_id': np.random.randint(0, n_users, n_transactions),
    'timestamp': sorted(timestamps),
    'amount': np.random.lognormal(3, 1.5, n_transactions).clip(0, 10000),
    'merchant_id': np.random.randint(0, 50, n_transactions),
    'hour_of_day': [t.hour for t in timestamps],
    'is_fraud': np.random.choice([0, 1], n_transactions, p=[0.95, 0.05])
})

# Add fraud patterns (higher amounts and unusual hours)
fraud_indices = df[df['is_fraud'] == 1].index
df.loc[fraud_indices, 'amount'] *= np.random.uniform(1.5, 3, len(fraud_indices))
df.loc[fraud_indices, 'hour_of_day'] = np.random.choice([2, 3, 4, 22, 23], len(fraud_indices))

print(f"Dataset shape: {df.shape}")
print(f"\nSample data:")
print(df.head(10))
print(f"\nFraud rate: {df['is_fraud'].mean()*100:.2f}%")

## 3. Velocity Features

Track transaction patterns over time windows (count, sum, mean)

In [ ]:
def compute_velocity_features(df, user_col='user_id', datetime_col='timestamp', 
                              amount_col='amount', time_windows=[(1, 'h'), (24, 'h')]):
    """
    Compute velocity features: transaction frequency and amounts over time windows.
    """
    df = df.copy()
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    df = df.sort_values([user_col, datetime_col])
    
    result = df.copy()
    
    for window_size, unit in time_windows:
        window_str = f"{window_size}{unit}"
        
        # Group by user and compute rolling statistics
        grouped = df.groupby(user_col, group_keys=False)
        
        # Transaction count in window
        rolling_count = grouped[amount_col].rolling(
            window=window_str, min_periods=1, on=datetime_col
        ).count()
        result[f"velocity_count_{window_str}"] = rolling_count.values
        
        # Total amount in window
        rolling_sum = grouped[amount_col].rolling(
            window=window_str, min_periods=1, on=datetime_col
        ).sum()
        result[f"velocity_sum_{window_str}"] = rolling_sum.values
        
        # Average amount in window
        rolling_mean = grouped[amount_col].rolling(
            window=window_str, min_periods=1, on=datetime_col
        ).mean()
        result[f"velocity_mean_{window_str}"] = rolling_mean.values
    
    # Time since last transaction
    df['prev_timestamp'] = df.groupby(user_col)[datetime_col].shift(1)
    result['velocity_time_since_last_s'] = (
        df[datetime_col] - df['prev_timestamp']
    ).dt.total_seconds()
    result['velocity_time_since_last_s'] = result['velocity_time_since_last_s'].fillna(999999)
    
    return result

# Apply velocity features
df_with_velocity = compute_velocity_features(df)

print("Velocity Features Added:")
velocity_cols = [c for c in df_with_velocity.columns if c.startswith('velocity_')]
print(velocity_cols)

print("\nSample velocity features:")
print(df_with_velocity[['user_id', 'timestamp', 'amount'] + velocity_cols].head(10))

## 4. Deviation Features

Compare current transaction to user's historical behavior

In [ ]:
def compute_deviation_features(df, user_col='user_id', features=['amount', 'hour_of_day'], window_size=30):
    """
    Compute deviation features: compare current to historical patterns.
    """
    df = df.copy()
    result = pd.DataFrame(index=df.index)
    
    for feature in features:
        # Compute per-user statistics
        user_stats = df.groupby(user_col)[feature].agg(
            mean='mean',
            std='std',
            q25=lambda x: x.quantile(0.25),
            q50=lambda x: x.quantile(0.50),
            q75=lambda x: x.quantile(0.75)
        )
        
        # Handle zero std
        user_stats['std'] = user_stats['std'].fillna(1.0)
        user_stats['std'] = user_stats['std'].replace(0, 1.0)
        
        # Merge stats
        df_merged = df.merge(user_stats, left_on=user_col, right_index=True, suffixes=('', '_hist'))
        
        # Compute z-score
        result[f"deviation_{feature}_zscore"] = (
            (df_merged[feature] - df_merged[f'{feature}_hist_mean']) / df_merged[f'{feature}_hist_std']
        )
        
        # Compute ratio
        result[f"deviation_{feature}_ratio"] = (
            df_merged[feature] / df_merged[f'{feature}_hist_mean'].replace(0, 1)
        )
    
    return result

# Apply deviation features
deviation_features = compute_deviation_features(df)

print("Deviation Features Added:")
deviation_cols = list(deviation_features.columns)
print(deviation_cols)

print("\nSample deviation features:")
print(pd.concat([df[['user_id', 'amount', 'hour_of_day']], deviation_features], axis=1).head(10))

## 5. Merchant Risk Features

Target encoding based on fraud rate per merchant

In [ ]:
def compute_merchant_risk_features(df, merchant_col='merchant_id', label_col='is_fraud', min_count=5):
    """
    Compute merchant risk features using target encoding.
    """
    df = df.copy()
    
    # Compute fraud rate per merchant
    merchant_stats = df.groupby(merchant_col)[label_col].agg(
        fraud_rate='mean',
        count='count'
    )
    
    # Smooth encoding (blend with global rate)
    global_fraud_rate = df[label_col].mean()
    smoothing = 10
    
    merchant_stats['fraud_rate_smoothed'] = (
        (merchant_stats['fraud_rate'] * merchant_stats['count'] + global_fraud_rate * smoothing) /
        (merchant_stats['count'] + smoothing)
    )
    
    # Merge back
    df['merchant_fraud_rate'] = df.merge(
        merchant_stats[['fraud_rate_smoothed']],
        left_on=merchant_col,
        right_index=True,
        how='left'
    )['fraud_rate_smoothed'].fillna(global_fraud_rate)
    
    # Risk bins
    df['merchant_risk_level'] = pd.cut(
        df['merchant_fraud_rate'],
        bins=[0, 0.01, 0.05, 1.0],
        labels=['Low', 'Medium', 'High']
    )
    
    return df

# Apply merchant risk features
df_with_merchant = compute_merchant_risk_features(df_with_velocity)

print("Merchant Risk Features Added:")
print(df_with_merchant[['merchant_id', 'merchant_fraud_rate', 'merchant_risk_level']].head(10))

print("\nRisk Level Distribution:")
print(df_with_merchant['merchant_risk_level'].value_counts())

## 6. Combine All Features

In [ ]:
# Combine all features
df_final = pd.concat([
    df_with_velocity.reset_index(drop=True),
    deviation_features.reset_index(drop=True)
], axis=1)

# Select feature columns
feature_cols = [
    'amount', 'hour_of_day',
    'velocity_count_1h', 'velocity_sum_1h', 'velocity_mean_1h',
    'velocity_count_24h', 'velocity_sum_24h', 'velocity_mean_24h',
    'velocity_time_since_last_s',
    'deviation_amount_zscore', 'deviation_amount_ratio',
    'deviation_hour_of_day_zscore',
    'merchant_fraud_rate'
]

# Drop rows with NaN
df_final = df_final.dropna(subset=feature_cols)

print(f"Final dataset shape: {df_final.shape}")
print(f"\nFeature columns ({len(feature_cols)}):")
print(feature_cols)

print("\nFeature statistics:")
print(df_final[feature_cols].describe())

## 7. Train Model with Engineered Features

In [ ]:
# Split data
X = df_final[feature_cols].values
y = df_final['is_fraud'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Random Forest
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("="*60)
print("MODEL PERFORMANCE WITH ENGINEERED FEATURES")
print("="*60)
print(f"\nAccuracy: {np.mean(y_pred == y_test)*100:.1f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, y_prob):.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Legitimate', 'Fraud']))

## 8. Feature Importance Analysis

In [ ]:
# Feature importance
importance = model.feature_importances_
indices = np.argsort(importance)[::-1]

# Plot
plt.figure(figsize=(12, 6))
plt.title('Feature Importance (Random Forest)', fontsize=14)
plt.bar(range(len(feature_cols)), importance[indices], color='steelblue', alpha=0.7)
plt.xticks(range(len(feature_cols)), [feature_cols[i] for i in indices], rotation=45, ha='right')
plt.xlabel('Feature', fontsize=12)
plt.ylabel('Importance', fontsize=12)
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
for i in indices[:5]:
    print(f"  {feature_cols[i]:.<40} {importance[i]:.3f}")

## 9. Summary

### Feature Engineering Techniques Covered:

1. **Velocity Features**: Track transaction patterns over time
   - Count, sum, mean over rolling windows
   - Time since last transaction

2. **Deviation Features**: Detect anomalous behavior
   - Z-score: (current - mean) / std
   - Ratio: current / historical_mean

3. **Merchant Risk Features**: Target encoding
   - Smoothed fraud rate per merchant
   - Risk level bins

### Key Takeaways:

- ✅ **Velocity features** capture burst patterns (high frequency = suspicious)
- ✅ **Deviation features** detect anomalies (large z-score = suspicious)
- ✅ **Merchant features** encode domain knowledge (risky merchants)
- ✅ **Feature importance** reveals what drives predictions

### Best Practices:

1. **Scale features**: Z-scores and ratios need different scales
2. **Handle missing**: Impute or drop NaN in rolling features
3. **Smooth encoding**: Blend target encoding with global rate
4. **Validate**: Check feature importance to avoid leakage

### Next Steps:
→ **Day 4**: Classification Benchmark (compare models)
→ **Day 5**: LSTM Autoencoder (sequence modeling)

---

**📁 Project Location**: `01_fraud_detection_core/fraud_feature_engineering/`